# Multi Source - Multi (Early-Exit) CNN Model --> Version to be Executed

## Parameters Definition

### Nodes

In [ ]:
import gurobipy as gp
from gurobipy import GRB

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

import graph as gg
import samples

import os

In [ ]:
# Parameters
C = 3  # Number of CNNs
N = 50  # Number of nodes
xm = 15  # Space width as (-xm, +xm)
dt = 7.5  # Transmission maximum distance
xs = None  # Source x position (None to randomly initialize)
ys = None  # Source y position (None to randomly initialize)
xf = None  # Sink x position (None to randomly initialize)
yf = None  # Sink y position (None to randomly initialize)
sink_is_source = True  # True if Source and Sink are on the same node

orange_p = 0.45  # Percentage of OrangePi Zero
beagle_p = 0.45  # Percentage of BeagleBone AI
pi3_p = 1 - orange_p - beagle_p  # Percentage of Raspberry Pi 3B+

# Max number of layers per node
L = 1
# Datarate (measured in KB/s --> default is 72.2 Mb/s == 9241.6 KB/s)
datarate = 9241.6
# Image Size (measured in KB --> default is a floating-point RGB image of size 227x227)
Ks = 227 * 227 * 3 * 4 / 1024

# Output dir
output_dir = 'results'

# Optimization Time Limit
time_limit = 300.0

exp_id = 0
# CNN name
cnn_name = 'alex'

In [ ]:
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

In [ ]:
ok = False
while not ok:
    # Create a graph object
    graph = gg.create_graph(nodes=N)
    # Create the nodes uniformly in the space
    xpos, ypos = gg.generate_random_positions(
        nodes=N,
        minx=-xm, maxx=xm,
        miny=-xm, maxy=xm
    )

    # Create the sources
    if xs is None:
        xs = np.random.rand(C) * 2 * xm - xm
        ys = np.random.rand(C) * 2 * xm - xm
        if sink_is_source:
            xf = xs
            yf = ys
        else:
            xf = np.random.rand(C) * 2 * xm - xm
            yf = np.random.rand(C) * 2 * xm - xm

    # Add the source to the graph
    xcoords = xpos.copy()
    ycoords = ypos.copy()
    for i in range(C):
        graph = gg.add_node_to_graph(graph)
        xcoords = np.append(xcoords, xs[i])
        ycoords = np.append(ycoords, ys[i])
    if not sink_is_source:
        for i in range(C):
            graph = gg.add_node_to_graph(graph)
            xcoords = np.append(xcoords, xf[i])
            ycoords = np.append(ycoords, yf[i])

    # Create the graph links among reachable nodes.
    graph = gg.add_arcs_with_max_distance(
        graph, xcoords, ycoords, dmax=dt
    )

    # Compute distances
    distances = np.zeros((N, N))
    for i in range(N):
        for j in range(i, N):
            dij = gg.find_min_distance(graph, i, j)
            distances[i, j] = dij
            distances[j, i] = dij
            
    # Create the distance from the source and to the sink
    source_dist = np.zeros((C, N))
    dest_dist = np.zeros((C, N))
    for c in range(C):
        for i in range(N):
            source_dist[c, i] = gg.find_min_distance(graph, i, N+c)
    if sink_is_source:
        dest_dist = source_dist
    else:
        for c in range(C):
            for i in range(N):
                dest_dist[c, i] = gg.find_min_distance(graph, i, N+C+c)
    
    if np.max(distances) < np.inf and np.max(source_dist) < np.inf and np.max(dest_dist) < np.inf:
        ok = True

In [ ]:
# Get configuration
cfg_m, cfg_c, cfg_e = samples.get_node_configuration('orange_beagle_pi3')
# Assign node configuration to each node
sequence = np.random.choice(
    np.arange(len(cfg_m)),
    size=N,
    replace=True,
    p=np.array([orange_p, beagle_p, pi3_p])
)

# Create the data structure for Gurobi
nodes, cbar, mbar, e = gp.multidict({
    i: [cfg_c[s], cfg_m[s], cfg_e[s]]
    for i, s in enumerate(sequence)
})

In [ ]:
# Save idx for metrics
orange_idx = np.argwhere(sequence==0).flatten()
beagle_idx = np.argwhere(sequence==1).flatten()
pi_idx = np.argwhere(sequence==2).flatten()

Finally, the just created network of sensors is visualized. In particular, the source and sink are represented by an orange star, the OrangePiZero nodes by the blue dots, the BeagleBone AI ones by the green diamonds, and the Raspberry Pi 3B+ by the purple crosses.

In [ ]:
# Plot
gg.plot(
    graph=graph, xpos=xcoords, ypos=ycoords, xm=xm, figsize=(15, 15),
    color=['blue', 'green', 'purple'], marker=['o', 'D', 'X'], size=[40, 80, 60],
    sequence=sequence, num_sources=C
)

### CNN 

In [ ]:
# Get connfiguration
cnn_k, cnn_m, cnn_e, cnn_p, cnn_g = samples.get_cnn_configuration_c(cnn_name)
# Create the data structure for Gurobi
layers, K, m, c, p, g = gp.multidict({
    i: [*cnn_params_i]
    for i, cnn_params_i in enumerate(zip(cnn_k, cnn_m, cnn_e, cnn_p, cnn_g))
}
)

Utility Functions

In [ ]:
def get_nodes_stats(alpha, list_idxs):
    """
    Function to counte the number of IoT units, per each type, used by each CNN.
    Please note that if a node is shared among two or more CNNs is counted two or more times.
    """
    stats = np.zeros((alpha.shape[0], len(list_idxs)))
    for u in range(alpha.shape[0]):
        au = alpha[u]
        for i, li in enumerate(list_idxs):
            stats[u, i] += np.sum(np.max(au[li], axis=1))
    return stats

In [ ]:
def get_processing_stats(alpha):
    stats = np.zeros(alpha.shape[0])
    for u in range(alpha.shape[0]):
        stats[u] = sum(
            [alpha[u, i, j] * p[j] * c[j] / e[i]
             for i in nodes for j in layers]
        )
    return stats

In [ ]:
def get_transmission_stats(alpha):
    stats = np.zeros(alpha.shape[0])
    for u in range(alpha.shape[0]):
        stats[u] = sum(
            [alpha[u, i, j] * alpha[u, k, j+1] * p[j+1] * distances[i, k] * K[j] / datarate 
             for i in nodes for k in nodes for j in layers[:-1]]
        ) + sum(
            [alpha[u, i, 1] * p[1] * Ks * source_dist[u, i] / datarate
             for i in nodes]
        ) + sum( 
            [alpha[u, i, j] * g[j] * K[len(layers)-1] * dest_dist[u, i] / datarate
             for i in nodes for j in layers]
        )
    return stats

## Model Definition

Once all the parameters have been defined, it is time to define the model.

In [ ]:
model = gp.Model('multi_ex_cnn_multi_source')

In [ ]:
# Save IoT stats
list_idx = [orange_idx, beagle_idx, pi_idx]
for i in range(len(list_idx)):
    np.savetxt(
        os.path.join(output_dir, 'node_{}_idxs_exp{}.csv'.format(i, exp_id)),
        list_idx[i], fmt='%.00f'
    )
# Test L from 1 to num_layers
M = len(cnn_m)
for L in range(M):
    alpha = model.addVars(C, len(nodes), len(layers), vtype=GRB.BINARY, name='alpha')
    
    # All the layers assigned.
    assigned_layer_constraints = \
        model.addConstrs((gp.quicksum(alpha[u, i, j] for i in nodes) == 1 
                          for j in layers for u in range(C)), name='assigned_layer')

    # Maximum number of layers per node
    layers_per_node_constraints = \
        model.addConstrs((gp.quicksum(alpha[u, i, j] for j in layers for u in range(C)) <= (L + 1)
                          for i in nodes), name='layers_per_node')

    # Computational Constraints
    # Note that c[u,j] is simplified in c[j] since all the CNNs are equal.
    computational_constraints = \
        model.addConstrs((gp.quicksum(alpha[u, i, j] * c[j] for j in layers for u in range(C)) <= cbar[i]
                          for i in nodes), name='computational_constraints')

    # Memory Constraints
    memory_constraints = \
        model.addConstrs((gp.quicksum(alpha[u, i, j] * m[j] for j in layers for u in range(C)) <= mbar[i]
                          for i in nodes), name='memory_constraints')
    
    # Transmission Time + Source Time + Sink Time + Processing Time
    # Note that K, p and g has no index u since all the CNNs are equal.
    model.setObjective(
        gp.quicksum(alpha[u, i, j] * alpha[u, k, j+1] * p[j+1] * distances[i, k] * K[j] / datarate 
                    for u in range(C) for i in nodes for k in nodes for j in layers[:-1]) +
        gp.quicksum(alpha[u, i, 1] * p[1] * Ks * source_dist[u, i] / datarate
                    for u in range(C) for i in nodes) + 
        gp.quicksum(alpha[u, i, j] * g[j] * K[len(layers)-1] * dest_dist[u, i] / datarate
                    for u in range(C) for i in nodes for j in layers) + 
        gp.quicksum(alpha[u, i, j] * p[j] * c[j] / e[i]
                    for u in range(C) for i in nodes for j in layers),
        GRB.MINIMIZE
    )
    
    # Max time_limit seconds of optimization
    model.setParam(GRB.Param.TimeLimit, time_limit)
    # Optimize
    model.optimize()
    
    # Convert the variables to numpy
    alpha_numpy = np.array([alpha[u, i, j].x for u in range(C) for i in nodes for j in layers]).reshape(C, len(nodes), len(layers))

    # Get objective value
    latency = model.getObjective().getValue()
    print('The total latency is {:.3f} seconds'.format(latency))
    
    # Compute and save stats
    nodes_stats = get_nodes_stats(
        alpha=alpha_numpy, list_idxs=[orange_idx, beagle_idx, pi_idx]
    )
    np.savetxt(
        os.path.join(output_dir, 'nodes_stats_exp{}_L{}.csv'.format(exp_id, L)),
        nodes_stats, fmt='%.00f'
    )
    processing_stats = get_processing_stats(
        alpha=alpha_numpy
    )
    np.savetxt(
        os.path.join(output_dir, 'processing_stats_exp{}_L{}.csv'.format(exp_id, L)),
        processing_stats, fmt='%.05f'
    )
    transmission_stats = get_transmission_stats(
        alpha=alpha_numpy
    )
    np.savetxt(
        os.path.join(output_dir, 'transmission_stats_exp{}_L{}.csv'.format(exp_id, L)),
        transmission_stats, fmt='%.05f'
    )
    for i in range(alpha_numpy.shape[0]):
        np.savetxt(
            os.path.join(output_dir, 'alpha_{}_exp{}_L{}.csv'.format(i, exp_id, L)),
            alpha_numpy[i], fmt='%.00f'
        )
    
    # PLOT
    # Plot the path per each CNN
    path_colors = ['darkred', 'orange', 'gray', 'black', 'olive', 'magenta']
    for u in range(C):
        # Find path
        path = np.argwhere(alpha_numpy[u].T==1)[:,1]
        if u==0:
            # First plot,
            ax = gg.path_plot(
                path=path, xpos=xcoords, ypos=ycoords, xm=xm, figsize=(15, 15), annotate=False,
                color=['blue', 'green', 'purple'], marker=['o', 'D', 'X'], size=[40, 80, 60],
                sequence=sequence, num_sources=C, path_source_idx=u,
                out_prob=g.values(), path_color=path_colors[u]
            )
        else:
            # Remaining plot, only paths
            gg.path_plot(
                path=path, xpos=xcoords, ypos=ycoords, xm=xm, figsize=(15, 15), annotate=False,
                color=['blue', 'green', 'purple'], marker=['o', 'D', 'X'], size=[40, 80, 60],
                sequence=sequence, num_sources=C, path_source_idx=u,
                out_prob=g.values(), path_color=path_colors[u], ax=ax, plot_background=False
            )
    # Small plt pause to avoid the plots are kept in memory and printed all together at the end.
    plt.pause(0.05)

    model.reset()